<a href="https://colab.research.google.com/github/fmathews11/College_Basketball_Functions/blob/main/Pulling_In_Boxscores.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from bs4 import BeautifulSoup
import numpy as np, seaborn as sns, matplotlib.pyplot as plt, warnings, time, requests, pandas as pd
warnings.filterwarnings("ignore")
from google.colab import output

In [2]:
ids = pd.read_csv('https://raw.githubusercontent.com/fmathews11/College_Basketball_Functions/main/ids.csv')
convert_dict = {'MIN':int,
               'OREB':int,
               'DREB':int,
               'REB':int,
               'AST':int,
               'STL':int,
               'BLK':int,
               'TO':int,
               'PF':int,
               'PTS':int,
               'FGM':int,
               'FGA':int,
               '3PM':int,
               '3PA':int,
               'FTM':int,
               'FTA':int}
col_order = ['Player',
                 'PTS',
                 'MIN',
                 'FGM',
                 'FGA',
                 '3PM',
                 '3PA',
                 'FTM',
                 'FTA',
                 'OREB',
                 'DREB',
                 'REB',
                 'AST',
                 'STL',
                 'BLK',
                 'TO',
                 'PF',
                 'Position']

def calculate_possessions(fga,orebs,tos,fta):
    value = (fga-orebs) + tos + (0.475*fta)
    return value

def clean_up_boxscore(df):
    df = df.iloc[:len(df)-2,]
    df['Position'] = [i[-1] for i in df.Starters]
    df = df.infer_objects()
    df.Starters = [i[:int((len(i)-1)/2)] for i in df.Starters]
    df['FGM'] = [i[0] for i in df.FG.str.split('-')]
    df['FGA'] = [i[1] for i in df.FG.str.split('-')]
    df['3PM'] = [i[0] for i in df['3PT'].str.split('-')]
    df['3PA'] = [i[1] for i in df['3PT'].str.split('-')]
    df['FTM'] = [i[0] for i in df.FT.str.split('-')]
    df['FTA'] = [i[1] for i in df.FT.str.split('-')]
    if type(df.MIN[0]) == str:
      df.MIN = 0
    df = df.astype(convert_dict)
    df = df.append(df.sum(numeric_only = True),ignore_index = True)
    last_row = len(df) - 1
    df = df.rename(columns = {"Starters":"Player"})
    df = df[col_order]
    df.iloc[last_row,0] = 'Team'
    df.iloc[last_row,17] = ""
    df['PTS/FGA'] = round((df.PTS/df.FGA),2).fillna(0)

    final_col_order = ['Player',
                'PTS',
                'MIN',
                'FGM',
                'FGA',
                '3PM',
                '3PA',
                'FTM',
                'FTA',
                'OREB',
                'DREB',
                'REB',
                'AST',
                'STL',
                'BLK',
                'TO',
                'PF',
                'PTS/FGA',
                 'Position']
    return df.astype(convert_dict)[final_col_order]

def get_boxscores(game_id,disp = False):

  #Create the URL and pull in the HTML
  url = 'https://www.espn.com/mens-college-basketball/boxscore/_/gameId/'+str(game_id)
  r = requests.get(url)
  soup = BeautifulSoup(r.content,'lxml')

  #Isolate the home team, away team, and game date.  Away team is always first
  away_team = str(soup.find('title')).split('-')[0].replace("<title>","").strip().split(' vs. ')[0]
  home_team = str(soup.find('title')).split('-')[0].replace("<title>","").strip().split(' vs. ')[1]
  game_date = str(soup.find("title")).split("-")[2].strip()

  #Infer tables with Pandas
  dfs = pd.read_html(url)
  #Away team is always index value 1, home team is 2
  away_team_df = dfs[1]
  home_team_df = dfs[2]
  # Convert columns from tuples to strings
  if type(away_team_df.columns[0]) is tuple:
    away_team_df.columns = [i[0] for i in away_team_df.columns.tolist()]
  if type(home_team_df.columns[0]) is tuple:
    home_team_df.columns = [i[0] for i in home_team_df.columns.tolist()]

  #Clean the dataframes
  away_team_df = clean_up_boxscore(away_team_df)
  home_team_df = clean_up_boxscore(home_team_df)
  #Create outer index
  away_team_df = pd.concat({away_team:away_team_df})
  home_team_df = pd.concat({home_team:home_team_df})

  if disp:
    display(away_team_df,home_team_df)
  else:
    return away_team_df,home_team_df

def get_agg_boxscore(game_id,disp = True):
    df_away,df_home = get_boxscores(game_id)
    away_team = df_away.index[0][0]
    home_team = df_home.index[0][0]
    last_row = len(df_away)
    df_away = df_away.iloc[last_row-1:]
    df_away = df_away[["PTS",
          "FGM",
          "FGA",
          "3PM",
          "3PA",
          "FTM",
          "FTA",
          "OREB",
          "DREB",
          "TO"]].reset_index().drop('level_1',1).rename(columns = {'level_0':'Team'}).set_index("Team")

    last_row = len(df_home)
    df_home = df_home.iloc[last_row-1:]
    df_home = df_home[["PTS",
          "FGM",
          "FGA",
          "3PM",
          "3PA",
          "FTM",
          "FTA",
          "OREB",
          "DREB",
          "TO"]].reset_index().drop('level_1',1).rename(columns = {'level_0':'Team'}).set_index("Team")
    away_team_dreb = df_away.DREB.item()
    home_team_dreb = df_home.DREB.item()
    df_home['OR%'] = 0
    df_away['OR%'] = 0
    if df_home.OREB.item() >0:
      df_home['OR%'] = 100*round(df_home.OREB.item()/(df_home.OREB.item() + away_team_dreb),2)
    if df_away.OREB.item() >0:
      df_away['OR%'] = 100*round(df_away.OREB.item()/(df_away.OREB.item() + home_team_dreb),2)
    final_df = pd.concat([df_away,df_home])

    final_df['POS'] = final_df.apply(lambda x: calculate_possessions(x.FGA,x.OREB,x.TO,x.FTA),axis = 1).astype(float).fillna(0)
    final_df['PTS_POS'] = round(final_df.PTS / final_df.POS,2).fillna(0)
    final_df['3PT%'] = 100*round(final_df['3PM'] / final_df['3PA'],2).fillna(0)
    final_df['FG%'] = 100*round(final_df.FGM/final_df.FGA,2).fillna(0)
    final_df["TS%"] = round(100*final_df.PTS/(2*(final_df.FGA + 0.475*final_df.FTA)),2).fillna(0)
    final_df['TO%'] = round(100*(final_df.TO/final_df.POS),2).fillna(0)
    final_df['POS'] = np.floor(final_df.POS).astype(int).fillna(0)
    type_dict = {'PTS':int,
                  'FGM':int,
                  'FGM':int,
                '3PM':int,
                '3PA':int,
                'FTM':int,
                'FTA':int,
                'OREB':int,
                'DREB':int,
                'TO':int,
                'POS':int}
    col_order = ['PTS',
                'FGM',
                'FGA',
                '3PM',
                '3PA',
                'FTM',
                'FTA',
                'OREB',
                'DREB',
                'TO',
                'POS',
                'PTS_POS',
                '3PT%',
                'FG%',
                'OR%',
                'TS%',
                'TO%']
    final_df = final_df[col_order]
    if disp:
      display(final_df.rename_axis("").astype(object).transpose())

    return final_df.astype(type_dict)

In [3]:
get_agg_boxscore(401364452)

,Rutgers,Purdue
PTS,72,84
FGM,27,26
FGA,53,49
3PM,5,8
3PA,15,22
FTM,13,24
FTA,17,29
OREB,6,5
DREB,16,19
TO,10,7


,PTS,FGM,FGA,3PM,3PA,FTM,FTA,OREB,DREB,TO,POS,PTS_POS,3PT%,FG%,OR%,TS%,TO%
Team,,,,,,,,,,,,,,,,,
Rutgers,72,27,53,5,15,13,17,6,16,10,65,1.11,33.0,51.0,24.0,58.94,15.37
Purdue,84,26,49,8,22,24,29,5,19,7,64,1.30,36.0,53.0,24.0,66.91,10.81


In [4]:
test_df = None
last_reported_time = 0

In [5]:
def plot_game_trends(test_df,half = 1,color1 = 'black',color2 = 'red'):
  melted = test_df.reset_index()
  for i in melted.columns.tolist()[:16]:
    plt.figure(figsize = (14,8))
    sns.lineplot(data = melted[melted.Half ==half].iloc[1:,:],
                x = 'Timestamp',
                y = i,
                hue = 'Team',
                palette = [color2,color1])
    plt.title(f"{melted.Team.tolist()[0]} vs {melted.Team.tolist()[1]}: {i.replace('_',' ')}")
    plt.ylabel(i.replace('_',' '))
    plt.xlabel('Time Remaining, 2nd Half')
    if half == 1:
      plt.xlabel('Time Remaining, 1st Half')
    plt.show();

In [6]:
def append_real_time_data(game_id,stats_df,disp = False):
  if disp:
    temp_df = get_agg_boxscore(401364432,disp = True)
  else:
    temp_df = get_agg_boxscore(401364432,disp = False)
  return pd.concat([stats_df,temp_df])
test_df = append_real_time_data(401364432,test_df)

In [7]:
def get_game_timestamp_half(game_id):
  """
  Returns a tuple:
    [0]: The current timestamp of the half currently being played
    [1]: Which half is currently being played
  """
  url = "https://www.espn.com/mens-college-basketball/playbyplay/_/gameId/"
  url = url + str(game_id)

  half = 2
  all_dfs = pd.read_html(url)
  if np.isnan(all_dfs[0].iloc[0,2]):
    half = 1
  
  target_df = df = pd.read_html(url)[1]
  current_time = df.iloc[0,0]
  return current_time,half
get_game_timestamp_half(401364432)

('20:00', 2)

In [32]:
def scrape_game_stats_real_time(game_id,max_retries = 14,glob = True):
  last_reported_timestamp = 0
  retry_iter = 0
  limbo_df = pd.DataFrame()

  if glob:
    global test_df
    while True:
      try:
        timestamp,half = get_game_timestamp_half(game_id)
      except:
        raise Exception("Game hasn't started")
        break
      if timestamp != last_reported_timestamp:
        last_reported_timestamp = timestamp
        output.clear()
        temp_df = get_agg_boxscore(game_id = game_id,disp = True)
        temp_df['Half'] = half
        temp_df['Timestamp'] = timestamp
        test_df = pd.concat([test_df,temp_df])
        test_df.to_csv("test_df.csv")
        get_boxscores(game_id,disp = True)
        retry_iter = 0
        time.sleep(np.random.randint(10,20))
      else:
        retry_iter +=1
        if retry_iter > max_retries:
          break
        print(f"Retry number {retry_iter} of {max_retries}")
        time.sleep(np.random.randint(15,22))
       
      
  
  else:
    while True:
      try:
        timestamp,half = get_game_timestamp_half(game_id)
      except:
        raise Exception("Game hasn't started")
        break
      if timestamp != last_reported_timestamp:
        last_reported_timestamp = timestamp
        output.clear()
        temp_df = get_agg_boxscore(game_id = game_id,disp = True)
        temp_df['Half'] = half
        temp_df['Timestamp'] = timestamp
        limbo_df = pd.concat([limbo_df,temp_df])
        limbo_df.to_csv("limbo_df.csv")
        get_boxscores(game_id,disp = True)
        time.sleep(np.random.randint(10,20))
      else:
        retry_iter +=1
        if retry_iter == max_retries:
          break
        print(f"Retry number {retry_iter} of {max_retries}")
        time.sleep(np.random.randint(20,25))
    return limbo_df


In [35]:
#test_df = pd.DataFrame()
scrape_game_stats_real_time(401364462)
test_df

,Purdue,Michigan State
PTS,65,68
FGM,25,27
FGA,48,57
3PM,1,9
3PA,9,20
FTM,14,5
FTA,19,10
OREB,4,5
DREB,22,17
TO,16,9


Player  PTS  MIN  FGM  FGA  ...  BLK  TO  PF  PTS/FGA  Position
Purdue 0      M. Gillis    0    0    0    2  ...    0   1   5     0.00         F
       1        Z. Edey   25    0   10   15  ...    2   3   4     1.67         C
       2        J. Ivey   16    0    5   10  ...    2   5   1     1.60         G
       3  E. Hunter Jr.    9    0    3    5  ...    0   2   0     1.80         G
       4  S. Stefanovic    2    0    1    6  ...    0   1   1     0.33         G
       5       C. Furst    2    0    1    1  ...    0   0   1     2.00         F
       6    T. Williams   11    0    5    9  ...    0   4   3     1.22         F
       7      E. Morton    0    0    0    0  ...    0   0   0     0.00         G
       8    I. Thompson    0    0    0    0  ...    0   0   0     0.00         G
       9           Team   65    0   25   48  ...    4  16  15     1.35          

[10 rows x 19 columns]

Player  PTS  MIN  FGM  ...  TO  PF  PTS/FGA  Position
Michigan State 0          M. Hall    2    0    1  ...   1   2     0.50         F
               1     J. Marble II   12    0    5  ...   2   3     2.40         F
               2         G. Brown   13    0    4  ...   0   0     1.44         F
               3      M. Christie   11    0    5  ...   0   1     1.10         G
               4       A. Hoggard   11    0    5  ...   3   3     0.92         G
               5   M. Bingham Jr.    2    0    1  ...   0   2     0.67         F
               6        J. Hauser    8    0    3  ...   0   1     1.60         F
               7       M. Sissoko    1    0    0  ...   1   2      inf         C
               8         J. Akins    0    0    0  ...   1   2     0.00         G
               9        T. Walker    8    0    3  ...   1   1     0.89         G
               10            Team   68    0   27  ...   9  17     1.19          

[11 rows x 19 columns]

Retry number 1 of 14
Retry number 2 of 14
Retry number 3 of 14
Retry number 4 of 14
Retry number 5 of 14
Retry number 6 of 14
Retry number 7 of 14
Retry number 8 of 14
Retry number 9 of 14
Retry number 10 of 14
Retry number 11 of 14
Retry number 12 of 14
Retry number 13 of 14
Retry number 14 of 14


KeyboardInterrupt: ignored

In [43]:
#plot_game_trends(test_df,half = 2,color1 = 'black',color2 = 'green')
import datetime
#pd.to_datetime(test_df.Timestamp)
datetime.datetime.strptime(test_df.Timestamp[0], "%M:%S")

datetime.datetime(1900, 1, 1, 0, 20)